In [1]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (21.8s)Resolving dependencies... (0.4s)Resolving dependencies... (4.0s)Resolving dependencies... (4.1s)Resolving dependencies... (5.1s)Resolving dependencies... (5.3s)Resolving dependencies... (11.4s)Resolving dependencies... (17.2s)Resolving dependencies... (20.1s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

BINANCE_SYMBOL = 'ETHUSDT'

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [3]:
target_period_hours = 3 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2022, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: Sat 10 Feb, 08:00 (71 hours left)
current price: 2361.71
-------------------------------
trading put 2325 and call 2375 (hit ratio: 53%)
cost: 0.017 / value: 0.035, benefit/cost = 2.0x
($) cost: 40.15 / value: 81.82, average gain = 41.67
-------------------------------
trading put 2300 and call 2400 (hit ratio: 47%)
cost: 0.011 / value: 0.027, benefit/cost = 2.4x
($) cost: 27.16 / value: 64.36, average gain = 37.20
-------------------------------
trading put 2275 and call 2425 (hit ratio: 41%)
cost: 0.007 / value: 0.021, benefit/cost = 3.0x
($) cost: 17.00 / value: 50.74, average gain = 33.74
-------------------------------
trading put 2250 and call 2450 (hit ratio: 34%)
cost: 0.005 / value: 0.017, benefit/cost = 3.7x
($) cost: 10.86 / value: 40.00, average gain = 29.13


[{'put': 2300.0, 'call': 2400.0, 'quantity': 1},
 {'put': 2275.0, 'call': 2425.0, 'quantity': 1}]

In [10]:
simulation.options_chain

,value_call,value_call_pct,call_ask,value_put,value_put_pct,put_ask
strike,,,,,,
2200.0,119.496607,NaN,NaN,11.417727,0.004947,0.0017
2225.0,98.182189,NaN,NaN,15.103309,0.006544,0.0026
2250.0,78.169593,NaN,NaN,20.090713,0.008705,0.0041
2275.0,60.035915,NaN,0.0235,26.957035,0.011680,0.0065
2300.0,44.676752,NaN,0.0140,36.597872,0.015857,0.0105
2325.0,33.083453,0.014334,0.0090,50.004572,NaN,0.0160
2350.0,24.656508,0.010683,0.0055,66.577628,NaN,NaN
2375.0,18.426109,0.007984,0.0029,85.347229,NaN,NaN
2400.0,13.820155,0.005988,0.0017,105.741275,NaN,NaN
